## 101 -- Compute Configs and Execution Environments in Anyscale

### Introduction

When you launch a Anyscale Workspace (or Jobs / Service), Anyscale deploys a Ray cluster using cloud resources according to your **cluster definition**. A **cluster definition** consists of the following two components:

- A [*container image*](https://docs.anyscale.com/configuration/configuration-overview#container-image) specifies the compute environment deployed to each node. It includes package requirements, dependencies, and environmental variables.
- A [*compute config*](https://docs.anyscale.com/configuration/configuration-overview#compute-config) specifies the number and type of Ray nodes to deploy, as well as instructions about scaling and various cloud-specific configuration options.

Workspaces, jobs, and services each have additional options and parameters to further customize clusters for an application.

In this notebook, we will cover the following:

- **Container Images** to standardize environments, manage dependencies, and configure runtime behavior  
- **Compute Configs** to specify cloud instance types, scaling rules, and advanced options like spot instances  
- **Updating a Anyscale Workspace** to use your custom compute configurations and container images  

### Creating a Container Image

Container images standardize the environment across all nodes in a cluster; bundling code, dependencies, and settings to ensure consistency across environments. On Anyscale, all created clusters (Workspaces, Jobs, or Services) run using a predefined container image.

Under the Configuration Tab on the left, click **Container Images.** You can view and filter from all existing container images. Click **Build** to configure a new container image.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config1.png" width="400"/>

In the configuration, give your Container Image a Name. Choose from the Anyscale Managed Images, copy the container image name and paste it in the config editor below. 

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config2.png" width="500"/>
<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config3.png" width="500"/>


Anyscale Managed base images come in multiple flavors:

1. **Slim images**: These images contain the minimum set of dependencies required to run on Anyscale, has the best performance
2. **Ray images**: These are the official [rayproject/ray](https://hub.docker.com/r/rayproject/ray) images
3. **Ray ML CPU images**
4. **Ray ML GPU images**: These are the GPU flavor of the [rayproject/ray-ml](https://hub.docker.com/r/rayproject/ray-ml) pipeline

After specifying your base image, you can pre-install packages and set environment variables by adding the following to your **config file, then click Build**

In [ ]:
# Set default model name environment variable
ENV MODEL_NAME=default-model

# Install pyproj Python package without cache
RUN pip install --no-cache-dir pyproj

# Test pyproj installation
RUN python -c "import pyproj; print('pyproj is successfully installed')"

Wait a few minutes while the container image is being built and registered.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config4.png" width="800"/>

### Creating a Compute Config

A compute configuration defines the virtual hardware used by a cluster. These resources shapes the cluster setup for running applications and workloads in a distributed computing environment.

Head to the Configuration section again, click on the **Compute Config** tab this time. **Click Create**

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config5.png" width="400"/>

Give your Compute Config a name. 

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config6.png" width="400"/>

Choose the Instance Type of the Head Node. 

The [Head node of a Ray cluster](https://docs.ray.io/en/latest/cluster/key-concepts.html#head-node) runs control processes which are responsible for cluster management. Anyscale enforces the following configurations to keep your head node healthy:

- For multi-node clusters, Ray workloads cannot be run on the head node
- For head-node only clusters, Ray workloads can be run on the head node. This is not recommended for production use cases

You can leave it as default or choose a CPU based machine.

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config7.png" width="600"/>

#### Configure the Worker Node(s)

In Ray, worker nodes are used to run the Ray workloads. 

1. For the **Worker Nodes**, choose a node instance type. These machines will run your Ray application, so choose the instance types and sizes (CPU or GPU) based on the needs of your workflow.
2. **Autoscaling**: Most workflows experience periods of bursty or spiky demand, Anyscale has the ability to autoscale clusters of nodes. Set the minimum number of nodes to 0 and the maximum to 3 to allow the cluster to scale based on workload.
3. **Spot Instances**: Under Advanced Config, you can enable Spot Instances to significantly reduce costs compared to on-demand machines, though they may be interrupted when cloud capacity is limited.
4. You can create **multiple node types.** Add another node type for GPU instances (T4 is enough). 
    - This is especially useful for Ray workflows that use CPU resources for data preprocessing and GPU acceleration for training or inference, allowing each task to run on the most cost-efficient and performant hardware.


<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config8.png" width="500"/>

Create the Compute Config. 


### Testing the Container Image and Compute Config with an Anyscale Workflow

Open up your existing **Anyscale Workspace.** that was created earlier.

Head over to node tab and edit the **Compute Configuration**. Update with your defined configuration and the cluster will restart. 

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config12.png" width="800"/>


Next, let's swap out the container image. Head over to **Depedencies** and edit the container image. Update the container to the one that you have created, **Save and Restart**

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config13.png" width="800"/>

Once started, click on the node button again to verify the compute configuration you have defined earlier. 

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config10.png" width="300"/>


Open a Jupyter Notebook in your Anyscale Workspace and run the following code in a cell. If it runs without errors, your container image, along with the environment variable and installed packages, has been successfully loaded. You can verify this in the **Depedencies** tab as well. 

<img src="https://lz-public-demo.s3.us-east-1.amazonaws.com/anyscale101/config11.png" width="500"/>

In [ ]:
import pyproj
import os

model_name = os.getenv("MODEL_NAME")
print(f"MODEL_NAME is: {model_name}")